In [1]:
import json
import numpy as np
class Graph:
    """
    Class for creating file congruence graph.
    Nodes representing users. Edge exists between nodes if users got common changed files. Edge color and width depends
    on number of common files between users.
    """

    def __init__(self, M, mult=1, heading="Graph"):
        self.M = M
        self.user_to_id = load_json(f"{resources}/userToId")
        self.id_to_user = {v: k for k, v in self.user_to_id.items()}
        self.mult = mult

        self.__init_net(heading)

    def __gen_quantiles(self):
        """
        Generate quantiles (0.25, 0.5, 0.75) for non null values
        """
        non_null_values = self.M[self.M != 0]
        self.quantiles = [
            np.quantile(non_null_values, 0.25)
            , np.quantile(non_null_values, 0.5)
            , np.quantile(non_null_values, 0.75)
        ]
        print(self.quantiles)

    def __init_net(self, heading):
        """
        Create net-graph
        :param heading: heading
        """
        self.net = Network(heading=heading
                           , height="1080px"
                           , width="100%"
                           , bgcolor="#222222"
                           , font_color="white")

        self.__gen_quantiles()

        self.net.barnes_hut()

        size = self.M.shape[0]
        for i in range(size):
            for j in range(size):
                if i == j:
                    continue
                    
                

                weight = int(self.M[i, j] * self.mult)
                if weight == 0:
                    continue
                src = self.id_to_user[i]
                dst = self.id_to_user[j]

#                 print(f"{src} -> {dst} = {weight}")
                self.net.add_node(src, src, title=src)
                self.net.add_node(dst, dst, title=dst)
                self.net.add_edge(src, dst, value=weight, color=self.edge_color(self.M[i, j], *self.quantiles))

        neighbor_map = self.net.get_adj_list()

        for node in self.net.nodes:
            node["title"] += " Neighbors:<br>" + "<br>".join(self.__pretty_string_neighbors(node, neighbor_map))
            node["value"] = len(neighbor_map[node["id"]])

    def __get_num_of_files_with_neighbor(self, user, neighbor):
        """
        Map function. Get number of common files with neighbor for user.
        :param user: user login
        :param neighbor: neighbor login
        :return: tuple (neighbor login, number of common files with neighbor)
        """
        num_of_files = self.M[self.user_to_id[user], self.user_to_id[neighbor]]
        return neighbor, num_of_files

    def __pretty_string_neighbors(self, node, neighbor_map):
        """
        Create string for better visualization.
        :param node: current node
        :param neighbor_map: neighbors for each node
        :return: string of neighbors and number of common files
        """
        result = map(lambda neighbor: self.__get_num_of_files_with_neighbor(node["title"], neighbor)
                     , neighbor_map[node["id"]])
        result = sorted(result, key=lambda x: x[1], reverse=True)
        result = map(lambda x: f"{x[0]} : {x[1]}", result)
        return result

    def show(self):
        self.net.show(f"{work_dir}/{self.net.heading}.html")

    @staticmethod
    def edge_color(weight, quantile1, quantile2, quantile3):
        if weight <= quantile1:
            return '#0569E1'
        if weight <= quantile2:
            return '#C1F823'
        if weight <= quantile3:
            return '#FCAA05'
        return '#EE5503'

In [3]:
def load_json(filename):
    with open(f"{filename}", 'r') as fp:
        data = json.load(fp)
    return data
def print_(M):
    print(f"min={M.min()}, mean={M.mean()}, max={M.max()}, shape={M.shape}")

In [4]:
resources = '/home/nikolaisv/study/intership/Analysis-of-Socio-Technical-Congruence/resources'
FDM = np.array(load_json(f"{resources}/fileDependencyMatrix"))
UFI = load_json(f"{resources}/userFilesIds")
AM = np.array(load_json(f"{resources}/assignmentMatrix"))

In [14]:
print_(AM[AM > 10])

min=11, mean=54.058465764175004, max=343, shape=(5097,)


In [11]:
non_null_values = AM[AM > 5]
quantiles = [
    np.quantile(non_null_values, 0.25)
    , np.quantile(non_null_values, 0.5)
    , np.quantile(non_null_values, 0.75)
]
print(quantiles)

[8.0, 13.0, 29.0]


In [4]:
AM_ = np.copy(AM)
threshold = 3
AM_[AM_ < threshold] = 0
AM_[AM_ >= threshold] = 1


In [ ]:
CN = AM_ @ FDM @ AM_.T

In [5]:
CN_n = CN / CN.max()
print_(CN_n)

min=0.0, mean=1.2778480925383054e-05, max=1.0, shape=(1681, 1681)


In [9]:
from pyvis.network import Network
work_dir = '/home/nikolaisv/study/intership/visualization/'

In [7]:
# CN_n[CN_n < 0.5] = 0
CN_ = np.copy(CN_n)
CN_[CN_ < 0.01] = 0
mult = 100
# graph = Graph(CN_, mult)
# graph.show()

In [6]:
import json
import numpy as np
class GraphGaps:
    """
    Class for creating file congruence graph.
    Nodes representing users. Edge exists between nodes if users got common changed files. Edge color and width depends
    on number of common files between users.
    """

    def __init__(self, D, A,  mult=1, heading="Graph"):
        self.D = D
        self.user_to_id = load_json(f"{resources}/userToId")
        self.id_to_user = {v: k for k, v in self.user_to_id.items()}
        self.Dult = mult
        self.A = A

        self.__init_net(heading)

    def __gen_quantiles(self):
        """
        Generate quantiles (0.25, 0.5, 0.75) for non null values
        """
        non_null_values = self.D[self.D != 0]
        self.quantiles = [
            np.quantile(non_null_values, 0.25)
            , np.quantile(non_null_values, 0.5)
            , np.quantile(non_null_values, 0.75)
        ]
        print(self.quantiles)

    def __init_net(self, heading):
        """
        Create net-graph
        :param heading: heading
        """
        self.net = Network(heading=heading
                           , height="1080px"
                           , width="100%"
                           , bgcolor="#222222"
                           , font_color="white")

        self.__gen_quantiles()

        self.net.barnes_hut()

        files = set()
        size = self.D.shape[0]
        for i in range(size):
            for j in range(size):
                if i == j:
                    continue
                    
                

                weight = int(self.D[i, j] * self.Dult)
                if weight == 0:
                    continue
                src = f"file{i}"
                dst = f"file{j}"
                files.add(src)
                files.add(dst)
#                 print(f"{src} -> {dst} = {weight}")
                self.net.add_node(src, src, title=src, color="red")
                self.net.add_node(dst, dst, title=dst, color="red")
                self.net.add_edge(src, dst, value=weight, color=self.edge_color(self.D[i, j], *self.quantiles))

#         for i in range(self.A.shape[0]):
#             for j in range(self.A.shape[1]):
#                 file_name = f"file{j}"
#                 if self.A[i, j] == 0:
#                     continue
#                 if file_name in files:
#                     node_name = f"user{i}"
#                     self.net.add_node(node_name, node_name, title=src, color="green")
#                     self.net.add_edge(node_name, file_name, value=5, color="green")
#         neighbor_map = self.net.get_adj_list()

#         for node in self.net.nodes:
#             node["title"] += " Neighbors:<br>" + "<br>".join(self.__pretty_string_neighbors(node, neighbor_map))
#             node["value"] = len(neighbor_map[node["id"]])

    def __get_num_of_files_with_neighbor(self, user, neighbor):
        """
        Map function. Get number of common files with neighbor for user.
        :param user: user login
        :param neighbor: neighbor login
        :return: tuple (neighbor login, number of common files with neighbor)
        """
#         num_of_files = self.D[self.user_to_id[int(user.replace("file", ""))], self.user_to_id[int(neighbor.replace("file", ""))]]
        return neighbor, self.D[int(neighbor)]

    def __pretty_string_neighbors(self, node, neighbor_map):
        """
        Create string for better visualization.
        :param node: current node
        :param neighbor_map: neighbors for each node
        :return: string of neighbors and number of common files
        """
        result = map(lambda neighbor: self.__get_num_of_files_with_neighbor(node["title"], neighbor.replace("file", ""))
                     , neighbor_map[node["id"]])
        result = sorted(result, key=lambda x: x[1], reverse=True)
        result = map(lambda x: f"{x[0]} : {x[1]}", result)
        return result

    def show(self):
        self.net.show(f"{work_dir}/{self.net.heading}.html")

    @staticmethod
    def edge_color(weight, quantile1, quantile2, quantile3):
        if weight <= quantile1:
            return '#0569E1'
        if weight <= quantile2:
            return '#C1F823'
        if weight <= quantile3:
            return '#FCAA05'
        return '#EE5503'

In [7]:
mult = 100
FDM_ = np.copy(FDM)
FDM_[FDM_ < 20] = 0 
FDM_[FDM_ >= 20] = 1
print_(FDM_[FDM_ == 1])

min=1, mean=1.0, max=1, shape=(1951942,)


In [ ]:
graph = GraphGaps(FDM_, AM_, mult)
graph.show()

[1.0, 1.0, 1.0]
